In [1]:
# pip install bertopic[spacy]

# installés en même temps :
# SentenceTransformer
# plotly
# (plotly pour visualizations de BERTopic)

In [2]:
import pandas as pd
# surveillance des GPU/RAm/etc.
import GPUtil
# vectorisation des textes
import torch
from sklearn.feature_extraction.text import CountVectorizer

# modèle
from bertopic import BERTopic
# error ou warning : c:\Users\ehess\anaconda3\envs\my4env\lib\site-packages\umap\distances.py:1063: 
# NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. 
# The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. 
# See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.@numba.jit()

c:\Users\ehess\anaconda3\envs\my4env\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\ehess\anaconda3\envs\my4env\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\ehess\anaconda3\envs\my4env\lib\site-packages\umap\distances.py:1086: NumbaDepr

In [3]:
import spacy

In [4]:
torch.cuda.is_available()


True

In [5]:
# pour un ordi avec GPU
is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    print("Using GPU!")
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
    print("GPU Usage")
    GPUtil.showUtilization()

Using GPU!
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  5% | 42% |


In [6]:
df = pd.read_csv("df_CC_lemmas_postag_corrected.csv", sep = ",", encoding = "utf-8", dtype= str)

# 20 sc

In [7]:
# df = df.sample(10000)

In [8]:
df["lemmas_without_stopwords"] = df["lemmas without stopwords"]
# la fonction .tolist() n'accepte pas les espace dans les noms de colonnes

In [9]:
lemmatized = df.lemmas_without_stopwords.tolist()
lemmatized[30]

"['augmenter', 'salaire', 'fonctionnaire', 'diminuer', 'taxe', 'service', 'revoir', 'convention', 'aide', 'domicile', 'prix', 'kilomètre', 'plupart', 'travailler', 'voiture', 'personnel', 'prix', 'proposer', 'prendre', 'charge', 'réalité', 'voiture', 'maire', 'm', 'besoin', 'entretien', 'pourtant', 'travail', 'domicile', 'permettre', 'accompagner', 'difficulté', 'social', 'malade', 'cadre', 'papier', 'administ', 'extérieure', 'prix', 'association', 'institution', 'temps', 'réfléchir', 'lire', 'comprendre', 'développement', 'ligne', 'ferroviaire', 'olorov', 'augmentation', 'horaire', 'cadencement', 'buzy', 'solution', 'permettre', 'diminuer', 'nombre', 'voiture', 'route', 'accidenter', 'gérer', 'préserver', 'environnement']"

In [10]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 | 12% | 41% |


In [11]:
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
# from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer


In [12]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [13]:
print(device)

cuda:0


In [14]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device = "cuda") # device = cuda ne génère pas d'erreur, il le trouve

In [15]:
# Step 1 - Extract embeddings

# the trained embedding produced with the La Javaness procedure, from CamemBert didn t work :
# embedding_model = SentenceTransformer(os.path.join('/Users/ehess/.cache/torch/sentence_transformers/dangvantuan_sentence-camembert-large/', 'sentence_bert_config.json'))
# '/Users/ehess/.cache/torch/sentence_transformers/dangvantuan_sentence-camembert-large'

# pour telecharger :
# from transformers.pipelines import pipeline
# embedding_model = pipeline("feature-extraction", model="dangvantuan/sentence-camembert-large")


In [17]:

# Step 1 - Extract embeddings


# embeddings = embedding_model.encode(lemmatized, show_progress_bar=True)
embeddings = embedding_model.encode(lemmatized, batch_size = 128, show_progress_bar=True)

# 56mn 8.6sc quand même


Batches:   0%|          | 0/1759 [00:00<?, ?it/s]

In [18]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  8% | 83% |


In [19]:
# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')


In [20]:

# Step 3 - Cluster reduced embeddings
# hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
hdbscan_model = HDBSCAN(min_cluster_size=30, metric='euclidean', cluster_selection_method='eom', prediction_data=True)



In [21]:
# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer()


In [22]:

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()


In [23]:
# Step 6 - (Optional) Fine-tune topic representations with a `bertopic.representation` model

# representation_model = KeyBERTInspired()

In [24]:
# initialisation de BERTopic. On choisit de suivre les conseils de Maartenger pour réduire la consommation de mémoire :  set low_memory to True, set calculate_probabilities to False
# All steps together
topic_model = BERTopic(
    low_memory = True, 
    calculate_probabilities = False,
    embedding_model=embedding_model,          # Step 1 - Extract embeddings
    umap_model=umap_model,                    # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
    verbose = True
)
# representation_model=representation_model # Step 6 - (Optional) Fine-tune topic represenations
# topic_model = topic_model.to(device)
# à suivre, il est aussi possible de "set the minimum frequency of words in the CountVectorizer class to reduce the size of the resulting sparse c-TF-IDF matrix." avec la commande suivante :
# vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english", min_df=10)
# The min_df parameter is used to indicate the minimum frequency of words. Setting this value larger than 1 can significantly reduce memory.

In [25]:
topics, probs = topic_model.fit_transform(lemmatized)

Batches:   0%|          | 0/7033 [00:00<?, ?it/s]

2023-05-17 16:31:24,021 - BERTopic - Transformed documents to Embeddings
2023-05-17 16:36:02,047 - BERTopic - Reduced dimensionality
2023-05-17 16:36:40,460 - BERTopic - Clustered reduced embeddings


In [26]:
topic_model.save("BERTopic_CC_model")

c:\Users\ehess\anaconda3\envs\my4env\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [7]:
# topic_model = BERTopic.load("BERTopic_CC_model")

In [27]:
# sauvegarde des topic en csv

freq = topic_model.get_topic_info()
print(freq.head(11))

    Topic   Count                                         Name
0      -1  150526           -1_retraite_pouvoir_devoir_salaire
1       0    6288         0_euro_retraite_pension_augmentation
2       1    6210          1_commune_rural_communauté_habitant
3       2    3880              2_école_élève_enfant_enseignant
4       3    3159  3_énergie_agriculture_écologique_transition
5       4    2693       4_augmentation_retraite_csg_indexation
6       5    2376            5_monsieur_président_macron_maire
7       6    1792            6_français_peuple_étranger_macron
8       7    1697               7_commune_rural_maire_habitant
9       8    1691              8_kilomètre_route_vitesse_heure
10      9    1664        9_élection_vote_blanc_proportionnelle


In [28]:
dd = [
    [item[0] if item[0] else pd.NA for item in topics]
    for topics in topic_model.get_topics().values()
]

ded = pd.DataFrame(dd)
ded

,0,1,2,3,4,5,6,7,8,9
0,retraite,pouvoir,devoir,salaire,faire,président,impôt,taxe,travail,payer
1,euro,retraite,pension,augmentation,retraité,csg,inflation,pourcent,indexation,mois
2,commune,rural,communauté,habitant,département,village,maire,région,local,service
3,école,élève,enfant,enseignant,éducation,classe,collège,scolaire,enseignement,parent
4,énergie,agriculture,écologique,transition,pesticide,pollution,production,électrique,nucléaire,environnement
...,...,...,...,...,...,...,...,...,...,...
348,humble,diminuer,ancien,avantage,président,sénateur,apercevoir,ministre,requête,député
349,pétition,identifiant,volet,mail,inscription,site,envoi,catégorisation,personnalisation,adresse
350,arrondissement,approfondi,dysfonctionnement,coordonnée,souhaiter,analyse,laisse,indique,informer,ordre
351,laïcité,renforcer,intangible,réciproque,religion,compréhension,engagement,application,garantir,hui


In [29]:
output = pd.concat([freq, ded], axis=1)

In [30]:
output.to_csv('size30_clusters_freq_BERTopic.csv', index=False)

In [31]:
fig = topic_model.visualize_barchart(top_n_topics=30)
fig.show()
fig.write_html("size30_barchart_topics.html")

In [39]:
# voir le détail des mots associés à un topic (defini par son num)
topic_model.get_topic(0)

[('commune', 0.008411323995234267),
 ('communauté', 0.00392758548546405),
 ('rural', 0.003717006240757502),
 ('habitant', 0.003274138798294203),
 ('communal', 0.002765976424006859),
 ('compétence', 0.0026164638492775687),
 ('département', 0.0025945814772607625),
 ('village', 0.002435490778005829),
 ('région', 0.0024035677482141782),
 ('maire', 0.0022275627761824904)]

In [32]:
# here i am reducing embeddings and creating a list of topics id for lowering the time of the map computation
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)


In [33]:
# map of the topics on similarity
first_100_topics = list(range(100))

figure_topic = topic_model.visualize_documents(lemmatized, topics = first_100_topics, reduced_embeddings=reduced_embeddings)

# Two options :

# Run the visualization with the original embeddings
# topic_model.visualize_documents(lemmatized, embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
# reduced_embeddings = reducer(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# topic_model.visualize_documents(lemmatized, embeddings)

In [34]:
figure_topic.write_html("30_size_cluster_figure_topic_docs.html")

In [49]:
# montre tous les topics et leurs mots associés. si default, la liste peut être longue.
topic_model.get_topics()

{-1: [('retraite', 0.00139889469648405),
  ('salaire', 0.0013407502237383245),
  ('pouvoir', 0.0012582627205297006),
  ('président', 0.0012550164023084098),
  ('devoir', 0.0012528234648936487),
  ('payer', 0.001252337967120741),
  ('taxe', 0.0012507004203832295),
  ('travail', 0.001250612832806649),
  ('suppression', 0.0012480021010681038),
  ('euro', 0.001246531196049288)],
 0: [('commune', 0.008411323995234267),
  ('communauté', 0.00392758548546405),
  ('rural', 0.003717006240757502),
  ('habitant', 0.003274138798294203),
  ('communal', 0.002765976424006859),
  ('compétence', 0.0026164638492775687),
  ('département', 0.0025945814772607625),
  ('village', 0.002435490778005829),
  ('région', 0.0024035677482141782),
  ('maire', 0.0022275627761824904)],
 1: [('école', 0.011452069582488487),
  ('élève', 0.005073102021365483),
  ('enseignant', 0.0038845987703248074),
  ('enfant', 0.0034342144022908466),
  ('éducation', 0.0033540911955555856),
  ('collège', 0.0032926581308575596),
  ('ensei

In [35]:
fig = topic_model.visualize_topics()
# fig.show()
fig.write_html("30_size_clusters_intertopic_distancemap_BERTopics.html")

In [36]:
first_100_topics = list(range(100))

In [38]:
# hierarchical_topics = topic_model.hierarchical_topics()


TypeError: BERTopic.hierarchical_topics() missing 1 required positional argument: 'docs'

In [40]:
hierarchie = topic_model.visualize_hierarchy()

In [41]:
hierarchie.show()